In [1]:
import cv2
import numpy as np

# Load YOLO model - Make sure to download these files
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load class labels (if your model is trained for parking spots)
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Function to detect parking spots in an image
def detect_parking_spots(image):
    height, width, channels = image.shape
    # Convert the image to a blob
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    # Loop over each detection
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Confidence threshold
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Non-max suppression to remove overlapping boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    available_spots = 0
    occupied_spots = 0

    # Iterate over detections and classify parking spots
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]

            # For simplicity, assuming label 'car' indicates an occupied spot
            if label == 'car':
                color = (0, 0, 255)  # Red for occupied
                occupied_spots += 1
            else:
                color = (0, 255, 0)  # Green for available
                available_spots += 1

            # Draw bounding box
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            cv2.putText(image, f"{label} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Display counts
    cv2.putText(image, f"Available Spots: {available_spots}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    cv2.putText(image, f"Occupied Spots: {occupied_spots}", (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    return image, available_spots, occupied_spots

# Process an image or video
def process_image_or_video(input_source):
    cap = cv2.VideoCapture(input_source)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detect parking spots in the current frame
        result_frame, available_spots, occupied_spots = detect_parking_spots(frame)

        # Show the result
        cv2.imshow("Parking Spot Detection", result_frame)

        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Main function to run the model
if __name__ == "__main__":
    # You can change 'video.mp4' to 0 to use the webcam or provide an image path
    process_image_or_video('parking_lot_video.mp4')  # Video or 'parking_image.jpg' for an image


error: OpenCV(4.10.0) /io/opencv/modules/dnn/src/darknet/darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: yolov3.cfg in function 'readNetFromDarknet'
